# Interactive Visualization Lab

Complete the following set of exercises to solidify your knowledge of interactive visualization using Plotly, Cufflinks, and IPyWidgets.

In [10]:
import pandas as pd
import chart_studio.plotly as py
import cufflinks as cf
from ipywidgets import interact

cf.go_offline()

In [13]:
data = pd.read_excel('../data/Online Retail.xlsx')

KeyboardInterrupt: 

## 1. Create an interactive bar chart showing total quantity and revenue by country (excluding United Kingdom) for the month of April 2011.

In [146]:
df = data[(data["Country"]!="United Kingdom")&(data["InvoiceDate"].dt.year==2011)&(data["InvoiceDate"].dt.month==4)]
df = df.groupby(df["Country"], as_index=False).sum()
df

,Country,InvoiceNo,Quantity,UnitPrice,Revenue,CustomerID
0,Australia,9338321,224,40.65,421.60,210681
1,Austria,13225072,308,95.18,584.78,298236
2,Belgium,60518251,1170,273.38,1788.48,1365606
3,Brazil,17606432,356,142.60,1143.60,408608
4,Channel Islands,4399312,96,28.10,243.00,119448
5,EIRE,164798924,4129,928.85,7270.50,4429510
6,Finland,19792080,810,67.98,1368.92,449182
7,France,119386873,2265,607.77,3899.31,2739646
8,Germany,231562735,5702,1263.74,10994.79,5306641
9,Greece,16479750,260,111.82,509.74,381510


In [147]:
import plotly.graph_objects as go
country=df['Country']

fig = go.Figure(data=[
    go.Bar(name='Quantity', x=df["Country"], y=df["Quantity"]),
    go.Bar(name='Revnue', x=df["Country"], y=df["Revenue"])
])
# Change the bar mode
fig.update_layout(barmode='group')
fig.show()

## 2. Create an interactive line chart showing quantity and revenue sold to France between January 1st and May 31st 2011.

In [205]:
df2 = data[(data["Country"]=="France")&(data["InvoiceDate"]<="2011.05.31")&(data["InvoiceDate"]>="2011.01.01")]
df2 = df2.drop(columns=["InvoiceNo", "UnitPrice", "CustomerID", "Description", "StockCode"])
df2 = df2.resample("D", on="InvoiceDate").sum().sort_values(by="InvoiceDate")
df2

,Quantity,Revenue
InvoiceDate,,
2011-01-05,728,1265.18
2011-01-06,438,709.02
2011-01-07,591,975.09
2011-01-08,0,0.00
2011-01-09,78,114.00
...,...,...
2011-05-23,304,385.96
2011-05-24,208,357.34
2011-05-25,734,1136.82


In [206]:
df2.iplot(kind="line")

## 3. Create an interactive scatter plot showing the relationship between average quantity (x-axis) and average unit price (y-axis) for the product PARTY BUNTING with the plot points color-coded by country (categories).

In [222]:
#x = quantity.mean
#y = unitprice.mean

df3 = data[data["Description"]=="PARTY BUNTING"]
df3 = df3.groupby("Country", as_index=False).agg({"Quantity":"mean", "UnitPrice":"mean"})
df3

,Country,Quantity,UnitPrice
0,Australia,33.125000,4.712500
1,Austria,8.000000,4.950000
2,Belgium,4.000000,4.950000
3,Channel Islands,13.333333,4.950000
4,Cyprus,2.333333,4.750000
5,Denmark,12.000000,4.950000
6,EIRE,21.210526,4.739474
7,Finland,6.000000,4.950000
8,France,5.727273,4.922727
9,Germany,6.800000,4.890000


In [223]:
df3.iplot(kind='scatter', mode='markers', x='Quantity', y='UnitPrice', categories="Country")

## 4. Create a set of interactive histograms showing the distributions of quantity per invoice for the following countries: EIRE, Germany, France, and Netherlands.

In [240]:
#d4 = data[(data["Country"]).str.contains("EIRE|Germany|France|Netherlands")]

lista = ["EIRE","Germany","France","Netherlands"]

for e in lista:
    d4 = data[data["Country"]==e]
    d4 = d4["Quantity"]
    print(d4.iplot(kind="hist"))

None


None


None


None


In [241]:
data

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country
0,536365,2010-12-01 08:26:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
1,536373,2010-12-01 09:02:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
2,536375,2010-12-01 09:32:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.55,15.30,17850,United Kingdom
3,536390,2010-12-01 10:19:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,64,2.55,163.20,17511,United Kingdom
4,536394,2010-12-01 10:39:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,32,2.55,81.60,13408,United Kingdom
...,...,...,...,...,...,...,...,...,...
396029,580691,2011-12-05 15:48:00,90214W,"LETTER ""W"" BLING KEY RING",12,0.29,3.48,13790,United Kingdom
396030,580691,2011-12-05 15:48:00,90214Z,"LETTER ""Z"" BLING KEY RING",12,0.29,3.48,13790,United Kingdom
396031,580865,2011-12-06 11:58:00,90089,PINK CRYSTAL SKULL PHONE CHARM,12,0.19,2.28,17914,United Kingdom
396032,580865,2011-12-06 11:58:00,90089,PINK CRYSTAL SKULL PHONE CHARM,12,0.19,2.28,17914,United Kingdom


## 5. Create an interactive side-by-side bar chart showing the revenue by country listed below (bars) for each of the products listed below.

In [242]:
product_list = ['JUMBO BAG RED RETROSPOT', 
                'CREAM HANGING HEART T-LIGHT HOLDER',
                'REGENCY CAKESTAND 3 TIER']

country_list = ['EIRE', 'Germany', 'France', 'Netherlands']

In [302]:
df5 = data[data.Description.str.contains('JUMBO BAG RED RETROSPOT|CREAM HANGING HEART T-LIGHT HOLDER|REGENCY CAKESTAND 3 TIER')]
df5 = df5[df5.Country.str.contains('EIRE|Germany|Netherlands|France')]
df5

,InvoiceNo,InvoiceDate,StockCode,Description,Quantity,UnitPrice,Revenue,CustomerID,Country,Year,Month,Day
179,539320,2010-12-16 19:16:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE,2010,12,16
198,539722,2010-12-21 13:45:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,24,2.95,70.80,14911,EIRE,2010,12,21
304,541570,2011-01-19 12:34:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,256,2.55,652.80,14646,Netherlands,2011,1,19
322,541979,2011-01-24 14:54:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE,2011,1,24
367,542777,2011-02-01 08:31:00,85123A,CREAM HANGING HEART T-LIGHT HOLDER,6,2.95,17.70,14911,EIRE,2011,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...
143798,580753,2011-12-06 10:00:00,22423,REGENCY CAKESTAND 3 TIER,1,12.75,12.75,12682,France,2011,12,6
143808,581098,2011-12-07 11:42:00,22423,REGENCY CAKESTAND 3 TIER,16,10.95,175.20,14911,EIRE,2011,12,7
143809,581175,2011-12-07 15:16:00,22423,REGENCY CAKESTAND 3 TIER,32,10.95,350.40,14646,Netherlands,2011,12,7
143810,581179,2011-12-07 15:43:00,22423,REGENCY CAKESTAND 3 TIER,64,10.95,700.80,12471,Germany,2011,12,7


In [303]:
df5 = df5.pivot_table(values="Revenue", columns="Description", index="Country", aggfunc="sum")

In [304]:
df5

Description,CREAM HANGING HEART T-LIGHT HOLDER,JUMBO BAG RED RETROSPOT,REGENCY CAKESTAND 3 TIER
Country,,,
EIRE,2740.80,278.72,7388.55
France,131.75,903.37,2816.85
Germany,35.40,1072.76,9061.95
Netherlands,1167.00,3468.00,3166.35


In [305]:
df5.iplot(kind="bar")

## 6. Create an interactive line chart showing quantity sold by day for the United Kingdom. Add drop-down boxes for Year and Month that allow you to filter the date range that appears in the chart.

In [328]:
'''
data['Year'] = pd.DatetimeIndex(data['InvoiceDate']).year
data['Month'] = pd.DatetimeIndex(data['InvoiceDate']).month
data['Day'] = pd.DatetimeIndex(data['InvoiceDate']).day
uk = data[data['Country']=='United Kingdom']
'''

In [322]:
df6 = uk[["Quantity","InvoiceDate"]]
data['Day']= df6.resample("D", on="InvoiceDate").sum().sort_values(by="InvoiceDate")
data['Month']= df6.resample("M", on="InvoiceDate").sum().sort_values(by="InvoiceDate")
data['Year']= df6.resample("Y", on="InvoiceDate").sum().sort_values(by="InvoiceDate")
uk = data[data['Country']=='United Kingdom']

,Quantity
InvoiceDate,
2010-12-01,21308
2010-12-02,30987
2010-12-03,7646
2010-12-04,0
2010-12-05,13603
...,...
2011-12-05,35874
2011-12-06,25909
2011-12-07,29078


In [335]:
@interact(Selection=['Day', 'Month', 'Year'])
def linechart(Selection):
    data = uk.pivot_table(values='Quantity', index=Selection).reset_index()

    data.iplot(kind='line', x=Selection, xTitle='bla', 
               yTitle='bla', title='BLA')

interactive(children=(Dropdown(description='Selection', options=('Day', 'Month', 'Year'), value='Day'), Output…

## 7. Create an interactive scatter plot that plots number of invoices (x-axis) vs. number of customers (y-axis) and the plot points represent individual products. Add two sliders that control the x and y axis ranges.

In [344]:
agg_func = {'InvoiceNo':'nunique',
            'Quantity':'sum',
            'UnitPrice':'mean',
            'Revenue':'sum',
            'CustomerID':'nunique'}

products = uk.groupby('Description').agg(agg_func)

In [353]:
products = products.reset_index()

In [357]:
products.iplot(x='InvoiceNo', y='CustomerID', 
               categories='Description', xTitle='esto', yTitle='lo_otro',
               title='bla')

## 8. Create an interactive bar chart that shows revenue by product description. Add a text field widget that filters the results to show the product that contain the text entered in their description.